# Import

In [23]:
import pandas as pd 
import plotly as px
pd.set_option('display.max_rows', 500)  # Zeilenanzahl erhöhen
pd.set_option('display.max_columns', 50)  # Spaltenanzahl erhöhen
pd.set_option('display.width', 1000)  # Maximale Breite der Anzeige



In [ ]:
years = [str(year) for year in range(2013, 2025)]
df = pd.read_excel("data.xlsx", sheet_name=years)
data_list = []
for year, df in df.items():
    # Füge das Jahr als neue Spalte hinzu
    df['Jahr'] = year
    
    # Füge den DataFrame der Liste hinzu
    data_list.append(df)

# Alle DataFrames in der Liste zu einem einzigen DataFrame zusammenführen
final_df = pd.concat(data_list, ignore_index=True)


In [27]:
df = final_df[final_df['Category']=='Anzahl Studierende'].drop(columns=['Variable', 'Category'])
df.columns

Index(['Bachelor Agrarwissenschaften', 'Bachelor Ernährungswissenschaften', 'Bachelor Nachwachsende Rohstoffe und Bioressourcen', 'Bachelor Ökotrophologie', 'Bachelor Umwelt und globaler Wandel', 'Master Ernährungswissenschaften ', 'Master  Ernährungsökonomie*', 'Master  Agrarökonomie und Betriebsmanagement', 'Master Nutztierwissenschaften', 'Master Pflanzenproduktion', 'Master Oenologie/Weinwirtschaft*', 'Master Haushalts- und Dienstleistungswissenschaften\n', 'Master Umwelt- und Ressourcenmanagement', 'Master Getränketechnologie', 'Jahr', 'Master Getränketechnologi', 'Master Agrar- und Ressourcenökonomie', 'Master Umweltwissenschaften', 'Master Transition Management\n', 'Master Ökotrophologie', 'Master Agrobiotechnologie\n', 'Master Insect Biotechnology and Bioresources', 'Bachelor Umweltmanagement', 'Master Agrobiotechnologie', 'Master Ernährungsökonomie', 'Master Nutzpflanzenwissenschaften', 'Master Oenologie/Weinwirtschaft', 'Master Transition Management',
       'Master Sustainab

# Lets make it in absolut numbers

### Lineplot

In [103]:
df_grouped = df.groupby(by='Jahr').sum().reset_index().sort_values(by='Jahr')

In [104]:
df_grouped.sort_values(by='Jahr')
columns_to_plot = df_grouped.columns.to_list()

In [115]:
import plotly.graph_objects as go

# Erstelle die Figure
fig = go.Figure()

# Füge jede Spalte manuell hinzu
fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Bachelor Agrarwissenschaften'],
                         mode='lines', name='Bachelor Agrarwissenschaften'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Bachelor Ernährungswissenschaften'],
                         mode='lines', name='Bachelor Ernährungswissenschaften'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Bachelor Nachwachsende Rohstoffe und Bioressourcen'],
                         mode='lines', name='Bachelor Nachwachsende Rohstoffe und Bioressourcen'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Bachelor Ökotrophologie'],
                         mode='lines', name='Bachelor Ökotrophologie'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Bachelor Umwelt und globaler Wandel'],
                         mode='lines', name='Bachelor Umwelt und globaler Wandel'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Ernährungswissenschaften '],
                         mode='lines', name='Master Ernährungswissenschaften '))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master  Ernährungsökonomie*'],
                         mode='lines', name='Master  Ernährungsökonomie*'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master  Agrarökonomie und Betriebsmanagement'],
                         mode='lines', name='Master  Agrarökonomie und Betriebsmanagement'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Nutztierwissenschaften'],
                         mode='lines', name='Master Nutztierwissenschaften'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Pflanzenproduktion'],
                         mode='lines', name='Master Pflanzenproduktion'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Oenologie/Weinwirtschaft*'],
                         mode='lines', name='Master Oenologie/Weinwirtschaft*'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Haushalts- und Dienstleistungswissenschaften\n'],
                         mode='lines', name='Master Haushalts- und Dienstleistungswissenschaften\n'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Umwelt- und Ressourcenmanagement'],
                         mode='lines', name='Master Umwelt- und Ressourcenmanagement'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Getränketechnologie'],
                         mode='lines', name='Master Getränketechnologie'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Getränketechnologi'],
                         mode='lines', name='Master Getränketechnologi'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Agrar- und Ressourcenökonomie'],
                         mode='lines', name='Master Agrar- und Ressourcenökonomie'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Umweltwissenschaften'],
                         mode='lines', name='Master Umweltwissenschaften'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Transition Management\n'],
                         mode='lines', name='Master Transition Management\n'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Ökotrophologie'],
                         mode='lines', name='Master Ökotrophologie'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Agrobiotechnologie\n'],
                         mode='lines', name='Master Agrobiotechnologie\n'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Insect Biotechnology and Bioresources'],
                         mode='lines', name='Master Insect Biotechnology and Bioresources'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Bachelor Umweltmanagement'],
                         mode='lines', name='Bachelor Umweltmanagement'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Agrobiotechnologie'],
                         mode='lines', name='Master Agrobiotechnologie'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Ernährungsökonomie'],
                         mode='lines', name='Master Ernährungsökonomie'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Nutzpflanzenwissenschaften'],
                         mode='lines', name='Master Nutzpflanzenwissenschaften'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Oenologie/Weinwirtschaft'],
                         mode='lines', name='Master Oenologie/Weinwirtschaft'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Transition Management'],
                         mode='lines', name='Master Transition Management'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Sustainable Transition*'],
                         mode='lines', name='Master Sustainable Transition*'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Agrar- und Ressourcenökonomie*'],
                         mode='lines', name='Master Agrar- und Ressourcenökonomie*'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Getränketechnologie*'],
                         mode='lines', name='Master Getränketechnologie*'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Informationstechnologie in den Agrar- und Umweltwissenschaften*'],
                         mode='lines', name='Master Informationstechnologie in den Agrar- und Umweltwissenschaften*'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Nachhaltige Ernährungswirtschaft'],
                         mode='lines', name='Master Nachhaltige Ernährungswirtschaft'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Sustainable Transition'],
                         mode='lines', name='Master Sustainable Transition'))

fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped['Master Nutztierwissenschaften*'],
                         mode='lines', name='Master Nutztierwissenschaften*'))




### Plot

In [116]:
fig.update_layout(
    width=2000,   # Breite in Pixel
    height=1000    # Höhe in Pixel
)

fig.show()

### iterativ solution -- does not work quite well 

In [101]:
# Bereinige die Spaltennamen von führenden und nachfolgenden Leerzeichen
df_grouped.columns = df_grouped.columns.str.strip()

# Dann den Plot nochmal generieren
fig = go.Figure()

# Iteriere durch die Spaltennamen und füge die Linien hinzu
for col in column_names:
    if col in df_grouped.columns:
        fig.add_trace(go.Scatter(x=df_grouped['Jahr'], y=df_grouped[col],
                                 mode='lines', name=col))
    else:
        print(f"Spalte {col} nicht gefunden")

# Zeige die Grafik an
fig.show()


Spalte Master Ernährungswissenschaften  nicht gefunden
Spalte Master Ernährungsökonomie* nicht gefunden
Spalte Master Agrarökonomie und Betriebsmanagement nicht gefunden
Spalte Master Haushalts- und Dienstleistungswissenschaften
 nicht gefunden
Spalte Master Transition Management
 nicht gefunden
Spalte Master Agrobiotechnologie
 nicht gefunden


### Heatmap

In [ ]:

df = df_grouped

# Um die Heatmap zu erstellen, nehmen wir die Spalten außer der "Jahr" Spalte:
heatmap_data = df.drop("Jahr", axis=1)

# Erstelle die Heatmap mit Plotly Express
fig = px.imshow(heatmap_data, 
                labels=dict(x="Studiengang", y="Jahr"), 
                x=heatmap_data.columns, 
                y=df["Jahr"], 
                color_continuous_scale='viridis')  # Du kannst die Farbskala ändern, z.B. 'Cividis', 'Blues', etc.
fig.update_layout(
    width=3000,   # Breite in Pixel
    height=1600    # Höhe in Pixel
)
fig.update_layout(
    xaxis=dict(
        title='Studiengänge',  # Titel der X-Achse (optional)
        tickfont=dict(size=20)  # Schriftgröße der Kategorien auf der X-Achse (Jahre)
    ),
    yaxis=dict(
        title='Jahre',  # Titel der Y-Achse (optional)
        tickfont=dict(size=20)  # Schriftgröße der Kategorien auf der Y-Achse (falls auch für Y-Achse relevant)
    ),
    coloraxis_colorbar=dict(
        tickfont=dict(size=20),
        len=0.7,  # Länge des Farbbalkens auf der Achse
    

))
# Zeige die Heatmap an
fig.show()
